# XGBoost

In [1]:
import sys, os
def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
        print("lib path is successfully appended.")
    else:
        print("lib path is already exists.")

importOwnLib()

import sonyalib as sonya
import importlib
importlib.reload(sonya)
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
import numpy as np

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier

lib path is successfully appended.


In [2]:
# fit random seed for reproductivity
seed = 7
np.random.seed(seed)

# Load metadata
file_path_train = './BRC_input_201116_train.xlsx' # train
file_path_test = './BRC_input_201116_test.xlsx' # test
meta_train = sonya.get_normalized_metadata(file_path_train)
meta_test = sonya.get_normalized_metadata(file_path_test)
properties = [
    'sex',
    'age',
    'LR',
    'cT',
    'cN',
    'cAverage',
    'cSD',
    'aAverage',
    'aSD',
    'lMax',
    'homogeneous',
    'hetero',
    'rim',
    'clustered',
    'non-mass',
    'AorCa',
    'LymAo',
    'LymCa'
]

num_properties = len(properties)

X_train = meta_train[properties]
y_train = meta_train['label']
X_test = meta_test[properties]
y_test = meta_test['label']

In [3]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (196, 18)
Training Labels Shape: (196,)
Testing Features Shape: (84, 18)
Testing Labels Shape: (84,)


In [7]:
xgb_clf = XGBClassifier(random_state=42)

print('Parameters currently in use: \n')
pprint(xgb_clf.get_params())

Parameters currently in use: 

{'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'objective': 'binary:logistic',
 'random_state': 42,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}


In [9]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 800, num = 8)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 5, stop = 30, num = 6)]
# Minimum number of samples required at each leaf node

# Create the random grid
xgb_param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth
                }

pprint(xgb_param_grid)

{'max_depth': [5, 10, 15, 20, 25, 30],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800]}


In [11]:
xgb_clf = XGBClassifier(random_state = 42)

XGB_gridSearch = GridSearchCV(estimator=xgb_clf,
                              param_grid = xgb_param_grid, 
                              scoring = 'accuracy', 
                              n_jobs = -1, 
                              cv = 5, 
                              verbose = 2, 
                              return_train_score = True)

XGB_gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=42,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, v

In [12]:
XGB_gridSearch.best_params_

{'max_depth': 5, 'n_estimators': 300}

In [13]:
xgb_clf_best = XGBClassifier(n_estimator = 300, 
                             max_depth = 5)

scoring = 'roc_auc'

score = cross_val_score(xgb_clf_best, X_train, y_train, cv = 10, n_jobs = -1, scoring = scoring)

score

array([0.91666667, 0.76190476, 0.89285714, 0.75      , 0.70238095,
       0.9047619 , 0.77142857, 0.8974359 , 0.87179487, 0.75641026])

In [15]:
from math import sqrt
x_bar = np.mean(score)
s = np.std(score)
n = len(score)

z = 1.96
lower = x_bar - (z * (s / sqrt(n)))
upper = x_bar + (z * (s / sqrt(n)))

print(s)
print(x_bar)
print(lower, upper)

0.07678711165319828
0.8225641025641026
0.7749709576812289 0.8701572474469763
